In [74]:
import json
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [83]:
!rm config.json  # önce eski dosyayı sil
!wget https://raw.githubusercontent.com/sisyphusinthegit/job-bot/main/config.json

--2025-07-10 15:06:43--  https://raw.githubusercontent.com/sisyphusinthegit/job-bot/main/config.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 708 [text/plain]
Saving to: ‘config.json’

config.json         100%[===================>]     708  --.-KB/s    in 0s      

2025-07-10 15:06:44 (41.9 MB/s) - ‘config.json’ saved [708/708]



In [84]:
config = load_config()
print(config["search_keywords"])

['data analyst', 'bi developer', 'data visualization', 'business intelligence', 'data', 'tableau', 'BI', 'Power BI']


In [85]:
def load_config(path="config.json"):
    with open(path, "r") as f:
        return json.load(f)

def send_email(subject, body):
    config = load_config()
    email_conf = config["email"]

    msg = MIMEMultipart()
    msg["From"] = email_conf["sender_email"]
    msg["To"] = email_conf["receiver_email"]
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    with smtplib.SMTP(email_conf["smtp_server"], email_conf["smtp_port"]) as server:
        server.starttls()
        server.login(email_conf["sender_email"], email_conf["sender_password"])
        server.send_message(msg)

    print("Email is sent successfully.")

In [86]:
def send_telegram(message):
  config = load_config()
  token = config["telegram"]["bot_token"]
  chat_id = config["telegram"]["chat_id"]

  url = f"https://api.telegram.org/bot{token}/sendMessage"
  payload = {
      "chat_id" : chat_id,
      "text": message
  }

  response = requests.post(url, data= payload)

  if response.status_code == 200:
    print("Message is sent successfully.")
  else:
    print(f"Error: {response.status_code} - {response.text}")


In [78]:
def notification(subject, message):
  try:
    send_email(subject, message)
  except Exception as excep:
    print(f"Email Error: {excep}")

  try:
    send_telegram(message)
  except Exception as excep:
    print(f"Telegram Error: {excep}")

In [79]:
notification("New job post", "New job arrived")

Email is sent successfully.
Message is sent successfully.


In [80]:
def remote_jobs():
    url = "https://remoteok.com/api"
    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        jobs = response.json()
        return jobs[1:]
    else:
        print(f"Hata: {response.status_code}")
        return []

In [ ]:
for i, job in enumerate(jobs):
    print(f"\nJob {i+1}")
    for key, value in job.items():
        print(f"{key}: {str(value)[:300]}")


Job 1
slug: remote-senior-flutter-developer-expertgrain-1093591
id: 1093591
epoch: 1752067450
date: 2025-07-09T13:24:10+00:00
company: ExpertGrain
company_logo: https://remoteOK.com/assets/img/jobs/8350d2ad14fbec6e824108d4a55618711752067450.jpg
position: Senior Flutter Developer
tags: ['front end']
logo: https://remoteOK.com/assets/img/jobs/8350d2ad14fbec6e824108d4a55618711752067450.jpg
description: <p class="MsoNormal" style="line-height:115%;margin:18pt 0cm 4pt 0cm;"><strong><span lang="en-ch" style="font-size:14pt;line-height:115%;font-family:Arial, sans-serif;" xml:lang="en-ch">ExpertGrain â Senior Flutter Developer</span></strong><strong><span lang="en-ch" style="font-size:14pt;line-heig
location: 
salary_min: 20000
salary_max: 40000
apply_url: https://remoteOK.com/remote-jobs/remote-senior-flutter-developer-expertgrain-1093591
original: True
url: https://remoteOK.com/remote-jobs/remote-senior-flutter-developer-expertgrain-1093591

Job 2
slug: remote-customer-onboarding-support

In [88]:
def relevant_job(job, config):
    title = job.get("position", "").lower()
    return any(kw.lower() in title for kw in config["search_keywords"])

    # Anahtar kelime eşleşmesi (pozisyon)
    if not any(kw.lower() in title for kw in config["search_keywords"]):
        return False

    return True

In [89]:
jobs = remote_jobs()
config = load_config()

for job in jobs:
    if relevant_job(job, config):
        message = f" {job['position']} @ {job.get('company', 'Unknown')}\n🌍 {job.get('location', 'Remote')}\n🔗 {job.get('url', '')}"
        notification("Yeni İş İlanı!", message)

Email is sent successfully.
Message is sent successfully.
Email is sent successfully.
Message is sent successfully.
Email is sent successfully.
Message is sent successfully.
Email is sent successfully.
Message is sent successfully.
Email is sent successfully.
Message is sent successfully.
Email is sent successfully.
Message is sent successfully.
Email is sent successfully.
Message is sent successfully.
Email is sent successfully.
Message is sent successfully.
